# Минипроект 4.
Итак, настало время провести свой первый A/B тест! 

## Задание
Представьте, что вы работаете аналитиком в очень крупной компании по доставке пиццы над приложением для курьеров (да, обычно в таких компаниях есть приложение для курьеров и отдельно приложение для потребителей).

У вас есть несколько ресторанов в разных частях города и целый штат курьеров. Но есть одна проблема, к вечеру скорость доставки падает из-за того, что курьеры уходят домой после рабочего дня, а количество заказов лишь растет. Это приводит к тому, что в момент пересмены наша доставка очень сильно проседает в эффективности. 

Наши data scientist-ы придумали новый алгоритм, который позволяет курьерам запланировать свои последние заказы перед окончанием рабочего дня так, чтобы их маршрут доставки совпадал с маршрутом до дома. То есть, чтобы курьеры доставляли последние свои заказы за день как бы "по пути" домой. 

Вы вместе с командой решили раскатить A/B тест на две равные группы курьеров. Часть курьеров использует старый алгоритм без опции "по пути", другие видят в своем приложении эту опцию и могут ее выбрать. Ваша задача – проанализировать данные эксперимента и помочь бизнесу принять решение о раскатке новой фичи на всех курьеров.
## Описание данных:
- order_id - id заказа
- delivery_time - время доставки в минутах
- district - район доставки
- experiment_group - экспериментальная группа

### Задание №1.
Для начала давайте сформулируем гипотезы для проверки. Выберите гипотезы, которые мы сможем проверить в этом эксперименте:
- Альтернативная гипотеза (H1): Разница между средним временем доставки в тестовой и контрольной группе есть
- Нулевая гипотеза (H0): Изменение алгоритма не влияет на скорость доставки
- Альтернативная гипотеза (H1): Разницы между средним временем доставки в тестовой и контрольной нет
- Альтернативная гипотеза (H1): Изменение алгоритма влияет на скорость доставки
- Нулевая гипотеза (H0): Разницы между средним временем доставки в тестовой и контрольной группе нет
- Нулевая гипотеза (H0): Разница между средним временем доставки в тестовой и контрольной группе есть

**Ответ:** 
- Альтернативная гипотеза (H1): Разница между средним временем доставки в тестовой и контрольной группе есть
- Нулевая гипотеза (H0): Разницы между средним временем доставки в тестовой и контрольной группе нет

### Задание №2.
Посчитайте количество наблюдений в тестовой и контрольной группе. На сколько по модулю отличается количество наблюдений в тесте и контроле?

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
couriers = pd.read_csv('C:/Users/79196/Projects/data/experiment_lesson_4.csv')
couriers

,order_id,delivery_time,district,experiment_group
0,3159,44.0,Purlieus,control
1,10426,51.0,Purlieus,control
2,9155,46.0,Purlieus,control
3,10699,34.0,Purlieus,control
4,10061,43.0,Purlieus,control
...,...,...,...,...
20191,14561,42.0,Suburb,test
20192,12015,32.0,Suburb,test
20193,20077,23.0,Purlieus,test
20194,14861,41.0,Central,test


In [3]:
count_nabl = couriers.groupby('experiment_group').agg({'order_id':'count'}).reset_index()
count_nabl

,experiment_group,order_id
0,control,10092
1,test,10104


**Ответ:** разница между контрольной и тестовой группой равна 12

### Задание №3.
Теперь давайте изучим наши распределения. Проверим выполнение условий на нормальность и посчитаем стандартное отклонение в тесте и в контроле.

Обязательно сделайте семплирование перед применением критерия при проверке на нормальность (и для теста, и для контроля), так как на больших выборках вы можете получать неверные результаты.

- Для того, чтобы проверить нормальность распределения, я использую SHAPIRO-WILK TEST
- тот тест показывает, что значения в тестовой группе распределены НОРМАЛЬНО
- В контрольной группе распределение является НОРМАЛЬНЫМ
- Стандартное отклонение времени доставки в тесте равно (округлите до сотых) 9.88
- Стандартное отклонение времени доставки в контроле равно (округлите до сотых) 9.99


In [4]:
sample_couriers_control = couriers[couriers['experiment_group'] == 'control']['delivery_time'].sample(1000, random_state=17)
sample_couriers_control

7144    42.0
6231    42.0
6902    41.0
7032    54.0
2009    44.0
        ... 
752     54.0
3754    63.0
5420    64.0
8119    49.0
4804    45.0
Name: delivery_time, Length: 1000, dtype: float64

In [5]:
sample_couriers_test = couriers[couriers['experiment_group'] == 'test']['delivery_time'].sample(1000, random_state=17)
sample_couriers_test

17877    37.0
18499    39.0
16476    48.0
14602    30.0
16719    25.0
         ... 
12410    40.0
18882    47.0
15944    47.0
14618    41.0
18816    36.0
Name: delivery_time, Length: 1000, dtype: float64

In [6]:
couriers.query('experiment_group == "control"').delivery_time.reset_index().std()

index            2913.453792
delivery_time       9.990017
dtype: float64

In [7]:
couriers.query('experiment_group == "test"').delivery_time.reset_index().std()

index            2916.917894
delivery_time       9.883308
dtype: float64

In [8]:
stats.shapiro(sample_couriers_control)

ShapiroResult(statistic=0.9974724054336548, pvalue=0.12408281117677689)

In [9]:
stats.shapiro(sample_couriers_test)

ShapiroResult(statistic=0.9975473880767822, pvalue=0.14001426100730896)

### Задание №4.
Пришло время сравнить средние в тесте и контроле. Внимание: здесь уже не нужно пользоваться методом .sample() - он нам нужен был только для теста Шапиро-Уилка, остальные тесты надо проводить на полных данных.

In [10]:
stats.ttest_ind(couriers.query('experiment_group == "test"').delivery_time, couriers.query('experiment_group == "control"').delivery_time)

Ttest_indResult(statistic=-43.03582663183714, pvalue=0.0)

**Ответ:**
- Для сравнения средних в данных экспериментальных группах я использую STUDENT'S T-TEST
- Статистика в тесте равна (округлите до целого) -43
- p-value < 0.05

### Задание №5.
На сколько процентов изменилось среднее время доставки в тестовой группе по сравнению с контрольной (округлите до сотых)? 

In [11]:
a_mean = couriers.query('experiment_group == "test"').delivery_time.mean()
a_mean

39.04681314330958

In [12]:
b_mean = couriers.query('experiment_group == "control"').delivery_time.mean()
b_mean

45.065101070154576

In [13]:
a_mean*100 / b_mean - 100

-13.354653121660817

**Ответ:** -13.35 %